In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import requests


In [2]:
if not os.path.exists('sales_textbook.txt'):
    url = 'https://github.com/861482002/little_gpt/blob/main/sales_textbook.txt'
    with open ('sales_textbook.txt','wb') as f:
        f.write(requests.get(url).connent)

In [3]:
with open('sales_textbook.txt','r') as f:
    text = f.read()

In [4]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
tokenized_text = encoding.encode(text)
len(tokenized_text)

# 放入tensor中
tokenized_text = torch.tensor(tokenized_text,dtype=torch.long)

In [5]:
# hyperparameters
context_length = 16
d_model = 64  # 512
batch_size = 4

In [6]:
train_idx= int(len(tokenized_text)*0.9)
train_text = tokenized_text[:train_idx]
valid_text = tokenized_text[train_idx:]

In [7]:
data = train_text
idxs = torch.randint(low = 0,high=len(data)-context_length,size=(batch_size,))
idxs
x_batch  = torch.stack([data[idx:idx+context_length] for idx in idxs])
x_batch.shape
y_batch  = torch.stack([data[idx+1:idx+context_length+1] for idx in idxs])

In [8]:
import pandas as pd
pd.DataFrame(x_batch[0].numpy())
encoding.decode(x_batch[0].numpy())


' and collaboration. By allowing customers to actively participate, they become more invested in the'

In [9]:
max_token = tokenized_text.max().item()
max_token
input_embedding_lookup_table = nn.Embedding(max_token+1, d_model)
input_embedding_lookup_table.weight.data
x_batch_embedding = input_embedding_lookup_table(x_batch)
y_batch_embedding = input_embedding_lookup_table(y_batch)
x_batch_embedding.shape

torch.Size([4, 16, 64])

In [10]:
# get positional encoding
import math
position_embedding_lookup_table = torch.zeros(context_length,d_model)
#self.position_embedding = torch.zeros(content_len, embed_dim).to('cuda')
position = torch.arange(0,context_length,dtype=torch.float).unsqueeze(1)

div_term = torch.exp(torch.arange(0, d_model, 2).float()*(-math.log(10000.0) / d_model))
position_embedding_lookup_table[:, 0::2] = torch.sin(position * div_term)
position_embedding_lookup_table[:, 1::2] = torch.cos(position * div_term)
position_embedding_lookup_table = position_embedding_lookup_table.unsqueeze(0).expand(batch_size,-1,-1)

In [11]:
x= x_batch_embedding+position_embedding_lookup_table
y= y_batch_embedding+position_embedding_lookup_table
pd.DataFrame(x[0].detach().numpy())

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.109855,2.343522,1.089280,-0.794655,0.142265,0.918729,-0.495731,0.144756,-2.095289,0.175440,...,-1.335010,2.429278,0.925689,1.457262,-0.373326,1.291208,1.052275,0.829475,0.868126,0.219483
1,-1.142227,-0.159050,1.153341,0.064835,-0.919469,-0.225585,0.035095,-0.064081,-0.282530,0.383314,...,-1.788772,0.065857,-0.968866,0.246873,-0.692178,1.643025,-1.428380,0.268652,-0.537004,1.627397
2,0.133297,0.531262,1.476049,-1.647408,0.029740,2.027748,-1.590934,1.074365,1.766964,0.876680,...,-0.629150,1.029477,-1.160042,1.602516,0.693355,2.032936,0.060755,-1.236737,0.682719,1.879789
3,1.493898,-3.389944,3.079340,-0.683920,1.456530,0.113880,0.918930,-0.184432,0.497643,1.152982,...,0.418869,1.745418,1.295429,-0.723709,-1.555816,1.450602,0.392184,1.037395,-0.456686,1.624979
4,-3.503409,-0.409884,-0.736673,-1.859448,1.331798,-0.707171,1.671194,0.762953,-0.037349,-1.871098,...,0.429432,0.562438,0.673450,-1.345922,0.068020,2.893505,0.311608,2.082561,1.087360,1.700012
5,-0.895758,0.491729,0.244283,-1.836137,0.070249,-0.981754,2.112711,-1.119058,2.590379,-0.704005,...,-0.211474,0.685010,-0.549047,1.226606,0.986982,-0.547893,-0.547215,3.490637,0.754871,0.483449
6,-0.681624,0.442755,-1.321213,-0.769429,-0.216595,-0.799868,-0.119360,-0.862467,0.050131,-0.092448,...,-0.114034,1.390097,-1.535013,0.682473,0.143339,1.599475,-0.076211,1.233852,-0.289764,0.423990
7,0.916014,3.469129,-1.416558,0.743983,-1.030558,-1.704276,0.976538,-0.918992,1.040372,-0.502018,...,-0.706066,-0.128519,1.940206,0.528680,0.582365,0.699189,-1.328603,1.448532,0.171255,0.818662
8,1.719478,0.372160,0.675517,-0.005417,-3.002513,0.669205,-1.104184,-0.962568,0.208409,-1.902654,...,-0.667853,2.640675,-0.789670,-0.666469,-1.355287,0.984543,1.491122,1.384979,-1.954079,2.203519
9,1.471005,-0.634261,0.024918,1.148145,-2.350325,-0.267630,0.331701,-0.119316,1.862687,-0.088233,...,-0.674907,2.295935,-0.062608,-0.128237,1.627358,2.534208,-0.655530,-0.296030,1.610565,2.269779


In [12]:
Wq = nn.Linear(d_model,d_model)# in 64,out 64 神经网络
Wk = nn.Linear(d_model,d_model)
Wv = nn.Linear(d_model,d_model)

Q=Wq(x) # 矩阵相乘？数据经过线性层，维度不变
K=Wq(x)
V=Wq(x)
Q.shape

torch.Size([4, 16, 64])

In [13]:
# apply multi head
num_heads=4
Q = Q.reshape(batch_size,context_length,num_heads,d_model//num_heads).permute(0,2,1,3)
Q.shape
K = K.reshape(batch_size,context_length,num_heads,d_model//num_heads).permute(0,2,1,3)
V = V.reshape(batch_size,context_length,num_heads,d_model//num_heads).permute(0,2,1,3)


In [14]:
output = Q @ K.transpose(-2,-1)/math.sqrt(d_model//num_heads) # Attention(Q,K,V)=softmax(QK^T)/sqrt(dkey)V

In [15]:
mask = torch.triu(torch.ones(context_length,context_length),diagonal=1).bool()
output = output.masked_fill(mask,float('-inf'))
pd.DataFrame(output[0,0].detach().numpy())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.370636,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
1,0.428245,0.956546,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
2,0.550198,0.693056,2.245313,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
3,0.452140,0.074772,-0.253837,1.803803,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
4,1.105765,0.293560,0.027906,1.231991,3.406918,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
5,0.369586,0.483948,-0.059424,0.520080,1.044584,2.765602,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
6,0.587927,0.092836,0.366165,0.175091,0.900850,0.752401,1.499416,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
7,0.109084,-0.050769,0.216869,-0.383277,-0.512337,-0.459565,0.021203,0.915395,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
8,0.258816,0.341590,0.032138,-0.200626,0.012507,1.167299,0.324670,0.255484,3.034547,-inf,-inf,-inf,-inf,-inf,-inf,-inf
9,0.581078,0.126215,1.111511,-0.081361,0.505267,0.346729,0.265252,0.140825,0.942706,2.235144,-inf,-inf,-inf,-inf,-inf,-inf


In [16]:
# apply softmax
attention_score = F.softmax(output,dim=-1)# 永远对最后一个维度进行
attention_score

tensor([[[[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.3709, 0.6291, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.1316, 0.1518, 0.7167,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0758, 0.0476, 0.0496,  ..., 0.2653, 0.0000, 0.0000],
          [0.0449, 0.0226, 0.1025,  ..., 0.0836, 0.3576, 0.0000],
          [0.0471, 0.0265, 0.0655,  ..., 0.0468, 0.0931, 0.2901]],

         [[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.4209, 0.5791, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.2451, 0.1489, 0.6061,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0576, 0.0553, 0.0365,  ..., 0.1266, 0.0000, 0.0000],
          [0.0412, 0.0251, 0.0610,  ..., 0.0544, 0.1974, 0.0000],
          [0.0556, 0.0572, 0.0489,  ..., 0.0618, 0.0601, 0.1011]],

         [[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.2454, 0.7546, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.2176, 0.2485, 0.5339,  ..., 0

In [17]:
# apply attention @v
A = attention_score @ V
A.shape # 批次，4个头，句子长度，每个头的维度


torch.Size([4, 4, 16, 16])

In [18]:
# apply concatenate 切好的头合并回来
A= A.transpose(1,2).reshape(batch_size,context_length,d_model)
A.shape
Wo= nn.Linear(d_model,d_model)
output = Wo(A)
output.shape

torch.Size([4, 16, 64])

In [19]:
# apply residual connection 残差连接
output = output+x

In [20]:
# apply layer normalization
layer_norm = nn.LayerNorm(d_model) # 使数据分布更加均匀
layer_norm_output = layer_norm(output)


In [23]:
# apply feed forward network
output = nn.Linear(d_model,d_model*4)(layer_norm_output)
output = nn.ReLU()(output)
output = nn.Linear(d_model*4,d_model)(output)

output = output+layer_norm_output

output.shape

torch.Size([4, 16, 64])

In [24]:
output = layer_norm(output)

In [26]:
# apply final linear layer
output = nn.Linear(d_model,max_token+1)(output)
output.shape


torch.Size([4, 16, 100070])

In [31]:
logits = F.softmax(output,dim=-1)
logits.shape
logits[0,0].sum()
predicted_index = torch.argmax(logits[0,0]).item()
encoding.decode([predicted_index])

'factory'